In [8]:
import requests
import time

def calculate_cost(api_key, location, radius, place_type):
    endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    places_count = 0
    next_page_token = None
    retries = 0
    max_retries = 3  # Maximum number of retries for fetching the next page
    page_count = 0  # Keep track of the number of pages processed

    while True:
        params = {
            'location': location,
            'radius': radius,
            'type': place_type,
            'key': api_key
        }

        if next_page_token:
            params['pagetoken'] = next_page_token
            print(f"Requesting page {page_count + 1} with next_page_token")
            time.sleep(2)  # Initial wait time before the token becomes valid

        response = requests.get(endpoint_url, params=params)
        data = response.json()

        if data['status'] == 'OK':
            places_count += len(data['results'])
            next_page_token = data.get('next_page_token')
            page_count += 1
            print(f"Page {page_count} fetched. Places found so far: {places_count}. Next page token: {next_page_token is not None}")

            if not next_page_token:
                print("No more pages to fetch.")
                break  # Exit loop if there's no next page
            retries = 0  # Reset retries for the next token
        elif data['status'] == 'INVALID_REQUEST' and retries < max_retries:
            print(f"Next page token not valid yet, retrying... Attempt {retries + 1}")
            time.sleep(3)  # Wait a bit longer for the token to become valid
            retries += 1
            continue  # Skip the rest of the loop and try again
        else:
            print(f"Failed to retrieve data or maximum retries reached: {data['status']}")
            break

    print(f"Total number of pages fetched: {page_count}")
    print(f"Total number of places found: {places_count}")

    cost_per_1000 = 17  # Assuming cost as per pricing information
    total_cost = (places_count / 1000) * cost_per_1000
    print(f"Estimated cost for fetching details of {places_count} places: ${total_cost:.2f}")

api_key = 'yourapikey'  # Replace with your actual API key
location = '33.75556,-84.40000'
radius = '5000'
place_type = 'restaurant'

calculate_cost(api_key, location, radius, place_type)


Page 1 fetched. Places found so far: 20. Next page token: True
Requesting page 2 with next_page_token
Page 2 fetched. Places found so far: 40. Next page token: True
Requesting page 3 with next_page_token
Page 3 fetched. Places found so far: 60. Next page token: False
No more pages to fetch.
Total number of pages fetched: 3
Total number of places found: 60
Estimated cost for fetching details of 60 places: $1.02


In [10]:
import requests
import pandas as pd
import time

# Your API Key and initial parameters
api_key = 'yourapikey'  # Replace with your actual API key
location = '33.75556,-84.40000'
radius = '5000'
type = 'restaurant'

# Function to get place details
def get_place_details(place_id, api_key):
    print(f"Fetching details for place ID: {place_id}")
    details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=formatted_address&key={api_key}"
    response = requests.get(details_url)
    details_data = response.json()
    if details_data.get('status') == 'OK':
        result = details_data.get('result', {})
        address = result.get('formatted_address', 'N/A')
    else:
        address = 'N/A'
    return address

# Function to fetch places with pagination support
def fetch_places(api_key, location, radius, type):
    places = []
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        'location': location,
        'radius': radius,
        'type': type,
        'key': api_key
    }

    page_count = 0
    while True:
        print(f"Requesting page {page_count + 1}")
        response = requests.get(url, params=params)
        data = response.json()

        if data['status'] == 'OK':
            # Process API response and collect place details
            for place in data['results']:
                place_id = place.get('place_id')
                name = place.get('name')
                lat = place.get('geometry', {}).get('location', {}).get('lat')
                lng = place.get('geometry', {}).get('location', {}).get('lng')
                address = get_place_details(place_id, api_key)  # Fetch place details
                places.append({'name': name, 'lat': lat, 'lng': lng, 'address': address})
                print(f"Added place: {name}")
            
            page_count += 1
            # Check for next page token
            next_page_token = data.get('next_page_token')
            if next_page_token:
                params['pagetoken'] = next_page_token
                time.sleep(2)  # Ensure the next_page_token is valid before proceeding
            else:
                print("No more pages to fetch.")
                break  # Exit loop if there's no next page token
        else:
            print(f"Failed to retrieve data: {data['status']}")
            break

    return places

# Fetch all places with pagination
places = fetch_places(api_key, location, radius, type)

# Create DataFrame
df = pd.DataFrame(places)

# Specify the file name (adjust the path as needed)
file_name = 'C:/Users/agutierrez/Downloads/places_data.xlsx'  # Adjusted for execution environment

# Export the DataFrame to an Excel file
df.to_excel(file_name, index=False)

print(f'Data saved to {file_name}')


Requesting page 1
Fetching details for place ID: ChIJWWUM7mQE9YgRuP3jMWQO-uI
Added place: The Varsity
Fetching details for place ID: ChIJgUolJWwE9YgRnxg5IduXOsg
Added place: Mary Mac's Tea Room
Fetching details for place ID: ChIJm8uX1X8E9YgR2Acbx-2xMZ4
Added place: STATS Brewpub
Fetching details for place ID: ChIJG8Wzy6gG9YgRBQVGToEnXTg
Added place: The Vortex
Fetching details for place ID: ChIJT9fzQXgE9YgRh3qCiYyrNJI
Added place: Hard Rock Cafe
Fetching details for place ID: ChIJr3-OMRsE9YgRNJTNacJKKqQ
Added place: TWO urban licks
Fetching details for place ID: ChIJvQqtX0ME9YgR-X6ukvNSt6I
Added place: South City Kitchen Midtown
Fetching details for place ID: ChIJFyduYFgP9YgRsb-T_vD8Fao
Added place: Maggiano's Little Italy
Fetching details for place ID: ChIJ86lMh3gE9YgRWFFqMrlgCXE
Added place: The Sun Dial Restaurant, Bar & View
Fetching details for place ID: ChIJK_An47MF9YgRbWKKF680tH4
Added place: SweetWater Brewing Company
Fetching details for place ID: ChIJQ-icamgE9YgR5x6mB7gMOR0
A